In [10]:
from  wsj_loader import *
import os
DATA_FOLDER_PATH = '../data'

In [2]:
wsj = WSJ()

In [3]:
train, test, dev = wsj.train, wsj.test, wsj.dev

In [9]:
print(train[0].shape, test[0].shape, dev[0].shape)
print(train[1].shape, test[1], dev[1].shape)

(24590,) (268,) (1103,)
(24590,) None (1103,)


In [135]:
def flatten_data(data=(), padding=20, freq_band=40, filename="", Limit=0):
    X = np.array([])
    Y = np.array([])
    Lookup = np.array([])
    curr_idx = 0
    IS_TEST_FILE = True if data[1] == None else False #File has only features no labels
    for idx in range(data[0].shape[0] if Limit == 0 else  Limit):
        print("Working on Frame number ",idx, " Total size with padding: ", X.shape[0])
        curr_idx += padding  
        frame = data[0][idx]
        
        padding_frame = np.zeros((padding,freq_band))
        padded_frame = np.concatenate((padding_frame, frame, padding_frame), axis=0)
        
        X = np.concatenate((X, padded_frame), axis=0) if X.size != 0 else padded_frame
        
        if IS_TEST_FILE == False:
            labels = data[1][idx]
            Y = np.concatenate((Y, labels), axis=0) if Y.size != 0 else labels
        
        idxes_range = np.array(range(idx, idx+frame.shape[0]))
        Lookup = np.concatenate((Lookup, idxes_range), axis=0) if Lookup.size != 0 else idxes_range
    
    np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_X"), arr=X)
    print("*****************Saved X file **********************")
    print("\tSize : ", X.shape)
    np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_Lookup"), arr=Lookup)
    print("*****************Saved Lookup file **********************")
    print("\tSize : ", Lookup.shape)
    if IS_TEST_FILE == False:
        np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_Y"), arr=Y)
        print("*****************Saved Labels file **********************")
        print("\tSize : ", Y.shape)
    del X, Lookup, Y
    return #X, Lookup, Y

In [138]:
%%time
flatten_data(test, filename="test")

*****************Saved X file **********************
	Size :  (180376, 40)
*****************Saved Lookup file **********************
	Size :  (169656,)
Wall time: 7.18 s


In [141]:
%%time
flatten_data(dev, filename="dev")

C:\Anaconda_3_5\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  


*****************Saved X file **********************
	Size :  (713414, 40)
*****************Saved Lookup file **********************
	Size :  (669294,)
*****************Saved Labels file **********************
	Size :  (669294,)
Wall time: 1min 53s


In [142]:
%%time
flatten_data(train, filename="train")

C:\Anaconda_3_5\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  


KeyboardInterrupt: 

In [149]:
def flatten_data_optimizer(data=(), padding=20, freq_band=40, filename="", Limit=0):
    curr_idx = 0
    
    IS_TEST_FILE = True if data[1] is None else False #File has only features no labels
    total_frames = sum([i.shape[0] for i in data[0]])
    num_padded_rows = total_frames + len(data[0])*2*padding # num_frames_utterance + 2*k*num_utterance
    
    X = np.zeros((num_padded_rows, freq_band))
    Y = np.zeros((total_frames))
    Lookup = np.zeros((total_frames))
    padding_frame = np.zeros((padding,freq_band))
    X_idx, Y_idx, LU_idx = 0, 0, 0 #not using LU_idx as Y_idx and LU_idx point to same  
    for idx in range(data[0].shape[0] if Limit == 0 else  Limit):
        print("Working on Frame number ",idx)
        curr_idx += padding  
        frame = data[0][idx]
        
        X_idx = X_idx + padding_frame.shape[0]
        X[X_idx: X_idx+frame.shape[0]] = frame
        X_idx = X_idx + padding_frame.shape[0]
        
        idxes_range = np.array(range(idx, idx+frame.shape[0]))
        Lookup[Y_idx: Y_idx + frame.shape[0]] = idxes_range

        if IS_TEST_FILE == False:
            labels = data[1][idx]
            Y[Y_idx: Y_idx + frame.shape[0]] = labels
        
        Y_idx = Y_idx + frame.shape[0]
        
    np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_X"), arr=X)
    print("*****************Saved X file **********************")
    print("\tSize : ", X.shape)
    np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_Lookup"), arr=Lookup)
    print("*****************Saved Lookup file **********************")
    print("\tSize : ", Lookup.shape)
    if IS_TEST_FILE == False:
        np.save(os.path.join(DATA_FOLDER_PATH, 'flattened', filename + "Flattened_Y"), arr=Y)
        print("*****************Saved Labels file **********************")
        print("\tSize : ", Y.shape)
    del X, Lookup, Y
    return #X, Lookup, Y

In [150]:
%%time
flatten_data_optimizer(data=dev, filename='dev')

Working on Frame number  0
Working on Frame number  1
Working on Frame number  2
Working on Frame number  3
Working on Frame number  4
Working on Frame number  5
Working on Frame number  6
Working on Frame number  7
Working on Frame number  8
Working on Frame number  9
Working on Frame number  10
Working on Frame number  11
Working on Frame number  12
Working on Frame number  13
Working on Frame number  14
Working on Frame number  15
Working on Frame number  16
Working on Frame number  17
Working on Frame number  18
Working on Frame number  19
Working on Frame number  20
Working on Frame number  21
Working on Frame number  22
Working on Frame number  23
Working on Frame number  24
Working on Frame number  25
Working on Frame number  26
Working on Frame number  27
Working on Frame number  28
Working on Frame number  29
Working on Frame number  30
Working on Frame number  31
Working on Frame number  32
Working on Frame number  33
Working on Frame number  34
Working on Frame number  35
Wo

Working on Frame number  424
Working on Frame number  425
Working on Frame number  426
Working on Frame number  427
Working on Frame number  428
Working on Frame number  429
Working on Frame number  430
Working on Frame number  431
Working on Frame number  432
Working on Frame number  433
Working on Frame number  434
Working on Frame number  435
Working on Frame number  436
Working on Frame number  437
Working on Frame number  438
Working on Frame number  439
Working on Frame number  440
Working on Frame number  441
Working on Frame number  442
Working on Frame number  443
Working on Frame number  444
Working on Frame number  445
Working on Frame number  446
Working on Frame number  447
Working on Frame number  448
Working on Frame number  449
Working on Frame number  450
Working on Frame number  451
Working on Frame number  452
Working on Frame number  453
Working on Frame number  454
Working on Frame number  455
Working on Frame number  456
Working on Frame number  457
Working on Fra

Working on Frame number  956
Working on Frame number  957
Working on Frame number  958
Working on Frame number  959
Working on Frame number  960
Working on Frame number  961
Working on Frame number  962
Working on Frame number  963
Working on Frame number  964
Working on Frame number  965
Working on Frame number  966
Working on Frame number  967
Working on Frame number  968
Working on Frame number  969
Working on Frame number  970
Working on Frame number  971
Working on Frame number  972
Working on Frame number  973
Working on Frame number  974
Working on Frame number  975
Working on Frame number  976
Working on Frame number  977
Working on Frame number  978
Working on Frame number  979
Working on Frame number  980
Working on Frame number  981
Working on Frame number  982
Working on Frame number  983
Working on Frame number  984
Working on Frame number  985
Working on Frame number  986
Working on Frame number  987
Working on Frame number  988
Working on Frame number  989
Working on Fra

In [151]:
%%time
flatten_data_optimizer(data=test, filename='test')

Working on Frame number  0
Working on Frame number  1
Working on Frame number  2
Working on Frame number  3
Working on Frame number  4
Working on Frame number  5
Working on Frame number  6
Working on Frame number  7
Working on Frame number  8
Working on Frame number  9
Working on Frame number  10
Working on Frame number  11
Working on Frame number  12
Working on Frame number  13
Working on Frame number  14
Working on Frame number  15
Working on Frame number  16
Working on Frame number  17
Working on Frame number  18
Working on Frame number  19
Working on Frame number  20
Working on Frame number  21
Working on Frame number  22
Working on Frame number  23
Working on Frame number  24
Working on Frame number  25
Working on Frame number  26
Working on Frame number  27
Working on Frame number  28
Working on Frame number  29
Working on Frame number  30
Working on Frame number  31
Working on Frame number  32
Working on Frame number  33
Working on Frame number  34
Working on Frame number  35
Wo

In [152]:
%%time
flatten_data_optimizer(data=train, filename='train')

Working on Frame number  0
Working on Frame number  1
Working on Frame number  2
Working on Frame number  3
Working on Frame number  4
Working on Frame number  5
Working on Frame number  6
Working on Frame number  7
Working on Frame number  8
Working on Frame number  9
Working on Frame number  10
Working on Frame number  11
Working on Frame number  12
Working on Frame number  13
Working on Frame number  14
Working on Frame number  15
Working on Frame number  16
Working on Frame number  17
Working on Frame number  18
Working on Frame number  19
Working on Frame number  20
Working on Frame number  21
Working on Frame number  22
Working on Frame number  23
Working on Frame number  24
Working on Frame number  25
Working on Frame number  26
Working on Frame number  27
Working on Frame number  28
Working on Frame number  29
Working on Frame number  30
Working on Frame number  31
Working on Frame number  32
Working on Frame number  33
Working on Frame number  34
Working on Frame number  35
Wo

Working on Frame number  532
Working on Frame number  533
Working on Frame number  534
Working on Frame number  535
Working on Frame number  536
Working on Frame number  537
Working on Frame number  538
Working on Frame number  539
Working on Frame number  540
Working on Frame number  541
Working on Frame number  542
Working on Frame number  543
Working on Frame number  544
Working on Frame number  545
Working on Frame number  546
Working on Frame number  547
Working on Frame number  548
Working on Frame number  549
Working on Frame number  550
Working on Frame number  551
Working on Frame number  552
Working on Frame number  553
Working on Frame number  554
Working on Frame number  555
Working on Frame number  556
Working on Frame number  557
Working on Frame number  558
Working on Frame number  559
Working on Frame number  560
Working on Frame number  561
Working on Frame number  562
Working on Frame number  563
Working on Frame number  564
Working on Frame number  565
Working on Fra

Working on Frame number  1040
Working on Frame number  1041
Working on Frame number  1042
Working on Frame number  1043
Working on Frame number  1044
Working on Frame number  1045
Working on Frame number  1046
Working on Frame number  1047
Working on Frame number  1048
Working on Frame number  1049
Working on Frame number  1050
Working on Frame number  1051
Working on Frame number  1052
Working on Frame number  1053
Working on Frame number  1054
Working on Frame number  1055
Working on Frame number  1056
Working on Frame number  1057
Working on Frame number  1058
Working on Frame number  1059
Working on Frame number  1060
Working on Frame number  1061
Working on Frame number  1062
Working on Frame number  1063
Working on Frame number  1064
Working on Frame number  1065
Working on Frame number  1066
Working on Frame number  1067
Working on Frame number  1068
Working on Frame number  1069
Working on Frame number  1070
Working on Frame number  1071
Working on Frame number  1072
Working on

Working on Frame number  1517
Working on Frame number  1518
Working on Frame number  1519
Working on Frame number  1520
Working on Frame number  1521
Working on Frame number  1522
Working on Frame number  1523
Working on Frame number  1524
Working on Frame number  1525
Working on Frame number  1526
Working on Frame number  1527
Working on Frame number  1528
Working on Frame number  1529
Working on Frame number  1530
Working on Frame number  1531
Working on Frame number  1532
Working on Frame number  1533
Working on Frame number  1534
Working on Frame number  1535
Working on Frame number  1536
Working on Frame number  1537
Working on Frame number  1538
Working on Frame number  1539
Working on Frame number  1540
Working on Frame number  1541
Working on Frame number  1542
Working on Frame number  1543
Working on Frame number  1544
Working on Frame number  1545
Working on Frame number  1546
Working on Frame number  1547
Working on Frame number  1548
Working on Frame number  1549
Working on

Working on Frame number  1953
Working on Frame number  1954
Working on Frame number  1955
Working on Frame number  1956
Working on Frame number  1957
Working on Frame number  1958
Working on Frame number  1959
Working on Frame number  1960
Working on Frame number  1961
Working on Frame number  1962
Working on Frame number  1963
Working on Frame number  1964
Working on Frame number  1965
Working on Frame number  1966
Working on Frame number  1967
Working on Frame number  1968
Working on Frame number  1969
Working on Frame number  1970
Working on Frame number  1971
Working on Frame number  1972
Working on Frame number  1973
Working on Frame number  1974
Working on Frame number  1975
Working on Frame number  1976
Working on Frame number  1977
Working on Frame number  1978
Working on Frame number  1979
Working on Frame number  1980
Working on Frame number  1981
Working on Frame number  1982
Working on Frame number  1983
Working on Frame number  1984
Working on Frame number  1985
Working on

...

Working on Frame number  22999
Working on Frame number  23000
Working on Frame number  23001
Working on Frame number  23002
Working on Frame number  23003
Working on Frame number  23004
Working on Frame number  23005
Working on Frame number  23006
Working on Frame number  23007
Working on Frame number  23008
Working on Frame number  23009
Working on Frame number  23010
Working on Frame number  23011
Working on Frame number  23012
Working on Frame number  23013
Working on Frame number  23014
Working on Frame number  23015
Working on Frame number  23016
Working on Frame number  23017
Working on Frame number  23018
Working on Frame number  23019
Working on Frame number  23020
Working on Frame number  23021
Working on Frame number  23022
Working on Frame number  23023
Working on Frame number  23024
Working on Frame number  23025
Working on Frame number  23026
Working on Frame number  23027
Working on Frame number  23028
Working on Frame number  23029
Working on Frame number  23030
Working 

Working on Frame number  23358
Working on Frame number  23359
Working on Frame number  23360
Working on Frame number  23361
Working on Frame number  23362
Working on Frame number  23363
Working on Frame number  23364
Working on Frame number  23365
Working on Frame number  23366
Working on Frame number  23367
Working on Frame number  23368
Working on Frame number  23369
Working on Frame number  23370
Working on Frame number  23371
Working on Frame number  23372
Working on Frame number  23373
Working on Frame number  23374
Working on Frame number  23375
Working on Frame number  23376
Working on Frame number  23377
Working on Frame number  23378
Working on Frame number  23379
Working on Frame number  23380
Working on Frame number  23381
Working on Frame number  23382
Working on Frame number  23383
Working on Frame number  23384
Working on Frame number  23385
Working on Frame number  23386
Working on Frame number  23387
Working on Frame number  23388
Working on Frame number  23389
Working 

Working on Frame number  23833
Working on Frame number  23834
Working on Frame number  23835
Working on Frame number  23836
Working on Frame number  23837
Working on Frame number  23838
Working on Frame number  23839
Working on Frame number  23840
Working on Frame number  23841
Working on Frame number  23842
Working on Frame number  23843
Working on Frame number  23844
Working on Frame number  23845
Working on Frame number  23846
Working on Frame number  23847
Working on Frame number  23848
Working on Frame number  23849
Working on Frame number  23850
Working on Frame number  23851
Working on Frame number  23852
Working on Frame number  23853
Working on Frame number  23854
Working on Frame number  23855
Working on Frame number  23856
Working on Frame number  23857
Working on Frame number  23858
Working on Frame number  23859
Working on Frame number  23860
Working on Frame number  23861
Working on Frame number  23862
Working on Frame number  23863
Working on Frame number  23864
Working 

Working on Frame number  24138
Working on Frame number  24139
Working on Frame number  24140
Working on Frame number  24141
Working on Frame number  24142
Working on Frame number  24143
Working on Frame number  24144
Working on Frame number  24145
Working on Frame number  24146
Working on Frame number  24147
Working on Frame number  24148
Working on Frame number  24149
Working on Frame number  24150
Working on Frame number  24151
Working on Frame number  24152
Working on Frame number  24153
Working on Frame number  24154
Working on Frame number  24155
Working on Frame number  24156
Working on Frame number  24157
Working on Frame number  24158
Working on Frame number  24159
Working on Frame number  24160
Working on Frame number  24161
Working on Frame number  24162
Working on Frame number  24163
Working on Frame number  24164
Working on Frame number  24165
Working on Frame number  24166
Working on Frame number  24167
Working on Frame number  24168
Working on Frame number  24169
Working 

Working on Frame number  24448
Working on Frame number  24449
Working on Frame number  24450
Working on Frame number  24451
Working on Frame number  24452
Working on Frame number  24453
Working on Frame number  24454
Working on Frame number  24455
Working on Frame number  24456
Working on Frame number  24457
Working on Frame number  24458
Working on Frame number  24459
Working on Frame number  24460
Working on Frame number  24461
Working on Frame number  24462
Working on Frame number  24463
Working on Frame number  24464
Working on Frame number  24465
Working on Frame number  24466
Working on Frame number  24467
Working on Frame number  24468
Working on Frame number  24469
Working on Frame number  24470
Working on Frame number  24471
Working on Frame number  24472
Working on Frame number  24473
Working on Frame number  24474
Working on Frame number  24475
Working on Frame number  24476
Working on Frame number  24477
Working on Frame number  24478
Working on Frame number  24479
Working 

In [154]:
15449191 + 24590*2*20

16432791